# Date and Time

## 5.1 Parsing a string into a timezone aware datetime object.

Python 3.2+ has support for %z format when parsing a string into a datetime object.

In [ ]:
import datetime
dt = datetime.datetime.strptime("2016-04-15T08:27:18-0500", "%Y-%m-%dT%H:%M:%S%z")

For other versions of Python, you can use an external library such as dateutil, which makes parsing a string with timezone into a datetime object is quick.

In [ ]:
import dateutil.parser
dt = dateutil.parser.parse("2016-04-15T08:27:18-0500")

The dt variable is now a datetime object with the following value:

In [ ]:
datetime.datetime(2016, 4, 15, 8, 27, 18, tzinfo=tzoffset(None, -18000))

## 5.2 Constructing timezone-aware datetimes

By default all datetime objects are naive. To make them timezone-aware, you must attach a tzinfo object, which
provides the UTC offset and timezone abbreviation as a function of date and time.

**Fixed Offset Time Zones**

For time zones that are a fixed offset from UTC, in Python 3.2+, the datetime module provides the timezone class, a concrete implementation of tzinfo, which takes a timedelta and an (optional) name parameter:

In [ ]:
from datetime import datetime, timedelta, timezone
JST = timezone(timedelta(hours=+9))
dt = datetime(2015, 1, 1, 12, 0, 0, tzinfo=JST)
print(dt)
# 2015-01-01 12:00:00+09:00
print(dt.tzname())
# UTC+09:00
dt = datetime(2015, 1, 1, 12, 0, 0, tzinfo=timezone(timedelta(hours=9), 'JST'))
print(dt.tzname)
# 'JST'

For Python versions before 3.2, it is necessary to use a third party library, such as dateutil. dateutil provides an
equivalent class, tzoffset, which (as of version 2.5.3) takes arguments of the form dateutil.tz.tzoffset(tzname,
offset), where offset is specified in seconds:

In [ ]:
from datetime import datetime, timedelta
from dateutil import tz
JST = tz.tzoffset('JST', 9 * 3600) # 3600 seconds per hour
dt = datetime(2015, 1, 1, 12, 0, tzinfo=JST)
print(dt)
# 2015-01-01 12:00:00+09:00
print(dt.tzname)
# 'JST'

**Zones with daylight savings time**

For zones with daylight savings time, python standard libraries do not provide a standard class, so it is necessary to use a third party library. pytz and dateutil are popular libraries providing time zone classes.

In addition to static time zones, dateutil provides time zone classes that use daylight savings time (see the
documentation for the tz module). You can use the tz.gettz() method to get a time zone object, which can then
be passed directly to the datetime constructor:

In [ ]:
from datetime import datetime
from dateutil import tz
local = tz.gettz() # Local time
PT = tz.gettz('US/Pacific') # Pacific time
dt_l = datetime(2015, 1, 1, 12, tzinfo=local) # I am in EST
dt_pst = datetime(2015, 1, 1, 12, tzinfo=PT)
dt_pdt = datetime(2015, 7, 1, 12, tzinfo=PT) # DST is handled automatically
print(dt_l)
# 2015-01-01 12:00:00-05:00
print(dt_pst)
# 2015-01-01 12:00:00-08:00
print(dt_pdt)
# 2015-07-01 12:00:00-07:00

**CAUTION**: As of version 2.5.3, dateutil does not handle ambiguous datetimes correctly, and will always default to the later date. There is no way to construct an object with a dateutil timezone representing, for example 2015-11-01 1:30 EDT-4, since this is during a daylight savings time transition.

All edge cases are handled properly when using pytz, but pytz time zones should not be directly attached to time zones through the constructor. Instead, a pytz time zone should be attached using the time zone's localize method:

In [ ]:
from datetime import datetime, timedelta
import pytz
PT = pytz.timezone('US/Pacific')
dt_pst = PT.localize(datetime(2015, 1, 1, 12))
dt_pdt = PT.localize(datetime(2015, 11, 1, 0, 30))
print(dt_pst)
# 2015-01-01 12:00:00-08:00
print(dt_pdt)
# 2015-11-01 00:30:00-07:00

Be aware that if you perform datetime arithmetic on a pytz-aware time zone, you must either perform the
calculations in UTC (if you want absolute elapsed time), or you must call normalize() on the result:

In [ ]:
dt_new = dt_pdt + timedelta(hours=3) # This should be 2:30 AM PST
print(dt_new)
# 2015-11-01 03:30:00-07:00
dt_corrected = PT.normalize(dt_new)
print(dt_corrected)
# 2015-11-01 02:30:00-08:00

## 5.3 Computing time differences

The timedelta module comes in handy to compute differences between times:

In [ ]:
from datetime import datetime, timedelta
now = datetime.now()
then = datetime(2016, 5, 23) # datetime.datetime(2016, 05, 23, 0, 0, 0)

Specifying time is optional when creating a new datetime object

In [ ]:
delta = now-then

delta is of type timedelta

In [ ]:
print(delta.days)
# 60
print(delta.seconds)
# 40826

To get n day's after and n day's before date we could use:

**n day's after date:**

In [ ]:
def get_n_days_after_date(date_format="%d %B %Y", add_days=120):
    date_n_days_after = datetime.datetime.now() + timedelta(days=add_days)
    return date_n_days_after.strftime(date_format)

**n day's before date:**

In [ ]:
def get_n_days_before_date(self, date_format="%d %B %Y", days_before=120):
    date_n_days_ago = datetime.datetime.now() - timedelta(days=days_before)
    return date_n_days_ago.strftime(date_format)

## 5.4 Basic datetime objects usage

The datetime module contains three primary types of objects - date, time, and datetime.

In [ ]:
import datetime
# Date object
today = datetime.date.today()
new_year = datetime.date(2017, 1, 1) #datetime.date(2017, 1, 1)
# Time object
noon = datetime.time(12, 0, 0) #datetime.time(12, 0)
# Current datetime
now = datetime.datetime.now()
# Datetime object
millenium_turn = datetime.datetime(2000, 1, 1, 0, 0, 0) #datetime.datetime(2000, 1, 1, 0, 0)

Arithmetic operations for these objects are only supported within same datatype and performing simple arithmetic with instances of different types will result in a TypeError.

In [ ]:
# subtraction of noon from today
noon-today
"""
Output:

Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.date'
However, it is straightforward to convert between types.
"""

# Do this instead
print('Time since the millenium at midnight: ',
    datetime.datetime(today.year, today.month, today.day) - millenium_turn)
# Or this
print('Time since the millenium at noon: ',
    datetime.datetime.combine(today, noon) - millenium_turn)

## 5.5 Switching between time zones

To switch between time zones, you need datetime objects that are timezone-aware.

In [ ]:
from datetime import datetime
from dateutil import tz

utc = tz.tzutc()
local = tz.tzlocal()

utc_now = datetime.utcnow()
utc_now # Not timezone-aware.

utc_now = utc_now.replace(tzinfo=utc)
utc_now # Timezone-aware.

local_now = utc_now.astimezone(local)
local_now # Converted to local time.

## 5.6 Simple date arithmetic

Dates don't exist in isolation. It is common that you will need to find the amount of time between dates or determine what the date will be tomorrow. This can be accomplished using timedelta objects.


In [ ]:
import datetime

today = datetime.date.today()
print('Today:', today)

yesterday = today - datetime.timedelta(days=1)
print('Yesterday:', yesterday)

tomorrow = today + datetime.timedelta(days=1)
print('Tomorrow:', tomorrow)

print('Time between tomorrow and yesterday:', tomorrow - yesterday)

"""
Output:

Today: 2016-04-15
Yesterday: 2016-04-14
Tomorrow: 2016-04-16
Difference between tomorrow and yesterday: 2 days, 0:00:00

"""

## 5.7 Converting timestamp to datetime

The datetime module can convert a POSIX timestamp to a ITC datetime object.

The Epoch is January 1st, 1970 midnight.

In [ ]:
import time
from datetime import datetime
seconds_since_epoch=time.time() #1469182681.709
utc_date=datetime.utcfromtimestamp(seconds_since_epoch) #datetime.datetime(2016, 7, 22, 10, 18, 1, 709000)

## 5.8 Substracting months from a date accurately

Using the calendar module

In [ ]:
import calendar
from datetime import date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, calendar.monthrange(y, m)[1])
    return date.replace(day=d,month=m, year=y)
next_month = monthdelta(date.today(), 1) #datetime.date(2016, 10, 23)

Using the dateutils module

In [ ]:
import datetime
import dateutil.relativedelta

d = datetime.datetime.strptime("2013-03-31", "%Y-%m-%d")
d2 = d - dateutil.relativedelta.relativedelta(months=1) #datetime.datetime(2013, 2, 28, 0, 0)

## 5.9 Parsing an arbitrary ISO 8601 timestamp with minimal libraries.

Python has only limited support for parsing ISO 8601 timestamps. For strptime you need to know exactly what format it is in. As a complication the stringification of a datetime is an ISO 8601 timestamp, with space as a separator and 6 digit fraction:

In [ ]:
str(datetime.datetime(2016, 7, 22, 9, 25, 59, 555555))
# '2016-07-22 09:25:59.555555'

#but if the fraction is 0, no fractional part is output

str(datetime.datetime(2016, 7, 22, 9, 25, 59, 0))
# '2016-07-22 09:25:59'

But these 2 forms need a different format for strptime. Furthermore, strptime' does not support at all
parsing minute timezones that have a:in it, thus2016-07-22 09:25:59+0300can be parsed, but the standard format2016-07-22 09:25:59+03:00` cannot.

There is a single-file library called iso8601 which properly parses ISO 8601 timestamps and only them.

It supports fractions and timezones, and the T separator all with a single function:

In [ ]:
import iso8601
iso8601.parse_date('2016-07-22 09:25:59')
# datetime.datetime(2016, 7, 22, 9, 25, 59, tzinfo=<iso8601.Utc>)
iso8601.parse_date('2016-07-22 09:25:59+03:00')
# datetime.datetime(2016, 7, 22, 9, 25, 59, tzinfo=<FixedOffset '+03:00' ...>)
iso8601.parse_date('2016-07-22 09:25:59Z')
# datetime.datetime(2016, 7, 22, 9, 25, 59, tzinfo=<iso8601.Utc>)
iso8601.parse_date('2016-07-22T09:25:59.000111+03:00')
# datetime.datetime(2016, 7, 22, 9, 25, 59, 111, tzinfo=<FixedOffset '+03:00' ...>)

If no timezone is set, iso8601.parse_date defaults to UTC. The default zone can be changed with default_zone keyword argument. Notably, if this is None instead of the default, then those timestamps that do not have an explicit timezone are returned as naive datetimes instead:

In [ ]:
iso8601.parse_date('2016-07-22T09:25:59', default_timezone=None)
# datetime.datetime(2016, 7, 22, 9, 25, 59)
iso8601.parse_date('2016-07-22T09:25:59Z', default_timezone=None)
# datetime.datetime(2016, 7, 22, 9, 25, 59, tzinfo=<iso8601.Utc>)

## 5.10 Get an ISO 8601 timestamp

* No timezone, microseconds

In [ ]:
from datetime import datetime
datetime.now().isoformat()
# Out: '2016-07-31T23:08:20.886783'

* Timezone, microseconds

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
datetime.now(tzlocal()).isoformat()
# Out: '2016-07-31T23:09:43.535074-07:00'

* Timezone, no microseconds

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
datetime.now(tzlocal()).replace(microsecond=0).isoformat()
# Out: '2016-07-31T23:10:30-07:00'

See: https://en.wikipedia.org/wiki/ISO_8601

## 5.11 Parsing a string with a short time zone name into a timezone aware datatime object

Using the dateutil library as in the previous example on parsing timezone-aware timestamps, it is also possible to parse timestamps with a specified "short" time zone name.

For dates formatted with short time zone names or abbreviations, which are generally ambiguous (e.g. CST, which could be Central Standard Time, China Standard Time, Cuba Standard Time, etc - more can be found here) or not necessarily available in a standard database, it is necessary to specify a mapping between time zone abbreviation and tzinfo object.

In [ ]:
from dateutil import tz
from dateutil.parser import parse
ET = tz.gettz('US/Eastern')
CT = tz.gettz('US/Central')
MT = tz.gettz('US/Mountain')
PT = tz.gettz('US/Pacific')
us_tzinfos = {'CST': CT, 'CDT': CT,
    'EST': ET, 'EDT': ET,
    'MST': MT, 'MDT': MT,
    'PST': PT, 'PDT': PT}
dt_est = parse('2014-01-02 04:00:00 EST', tzinfos=us_tzinfos)
dt_pst = parse('2016-03-11 16:00:00 PST', tzinfos=us_tzinfos)

After running this:

In [ ]:
dt_est
# datetime.datetime(2014, 1, 2, 4, 0, tzinfo=tzfile('/usr/share/zoneinfo/US/Eastern'))
dt_pst
# datetime.datetime(2016, 3, 11, 16, 0, tzinfo=tzfile('/usr/share/zoneinfo/US/Pacific'))

It is worth noting that if using a pytz time zone with this method, it will not be properly localized:

In [ ]:
from dateutil.parser import parse
import pytz
EST = pytz.timezone('America/New_York')
dt = parse('2014-02-03 09:17:00 EST', tzinfos={'EST': EST})

This simply attaches the pytz time zone to the datetime:

In [ ]:
dt.tzinfo # Will be in Local Mean Time!
# <DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

If using this method, you should probably re-localize the naive portion of the datetime after parsing:

In [ ]:
dt_fixed = dt.tzinfo.localize(dt.replace(tzinfo=None))
dt_fixed.tzinfo # Now it's EST.
# <DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

## 5.12 Fuzzy datetime parsing (extracting atetime out of a text)

It is possible to extract a date out of a text using the dateutil parser in a "fuzzy" mode, where components of the string not recognized as being part of a date are ignored.

In [ ]:
from dateutil.parser import parse

dt = parse("Today is January 1, 2047 at 8:21:00AM", fuzzy=True)
print(dt)

dt is now a datetime object and you would see datetime.datetime(2047, 1, 1, 8, 21) printed.

## 5.13 Iterate over dates

Sometimes you want to iterate over a range of dates from a start date to some end date. You can do it using
datetime library and timedelta object:

In [ ]:
import datetime

# The size of each step in days
day_delta = datetime.timedelta(days=1)

start_date = datetime.date.today()
end_date = start_date + 7*day_delta

for i in range((end_date - start_date).days):
    print(start_date + i*day_delta)

"""
Which produces:

2016-07-21
2016-07-22
2016-07-23
2016-07-24
2016-07-25
2016-07-26
2016-07-27
"""